In [8]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import pandas as pd
from flaml import AutoML
from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm
from xgboost import XGBClassifier

from util import engineer_features, prep_X_y

DATA_DIR = Path("./pistachio_1_data")
dyads_df = pd.read_csv(DATA_DIR / "all_dyads.csv")

sorted_dyads_df = dyads_df.sort_values(
    by="ActivityDateTime", key=lambda x: pd.to_datetime(x)
)
cleaned_dyads_dfs = engineer_features(
    sorted_dyads_df,
    stress_lookback_days=0,
    sleep_days_to_keep=[1, 2],
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/var/folders/3f/rygbfpzx3l7fcpbc7540q88w0000gr/T/ipykernel_91067/740823911.py:15: DtypeWarning: Columns (95,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  dyads_df = pd.read_csv(DATA_DIR / "all_dyads.csv")
/Users/zhu.henian/projects/pistachio/util.py:135: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["stress_avg_garmin_0_to_15m"] = df.pop("StressLevelValueAverage")


In [15]:
import optuna
from sklearn.model_selection import GroupKFold

feature_sets = [
    "hr",
    "activity",
    "sleep",
    "stress",
    "overnight_hrv",
    "medical",
    "therapy",
    "child_demo",
    "parent_demo",
    "temporal",
]


def objective(trial: optuna.Trial) -> float:
    selected = []
    for i, fs in enumerate(feature_sets):
        if trial.suggest_categorical(fs, [True, False]):
            selected.append(fs)
    # Always include "hr" and "activity"
    fs_subset = selected

    combined_df = pd.concat(
        [
            cleaned_dyads_dfs["index"],
            cleaned_dyads_dfs["response"],
        ]
        + [cleaned_dyads_dfs[fs] for fs in fs_subset],
        axis=1,
    )

    window = "30m"
    X_all, y_all = prep_X_y(combined_df, f"tantrum_within_{window}")

    automl_settings = {
        "time_budget": 5,  # seconds
        # "train_time_limit": 1,  # seconds
        "task": "classification",
        "metric": "log_loss",
        "estimator_list": ["xgboost"],
        # "split_type": time_series_split,
        "early_stop": True,
        "verbose": False,
    }
    automl = AutoML()
    automl.fit(X_all, y_all, **automl_settings)

    group_kfold = GroupKFold(n_splits=5)
    groups = combined_df["dyad"]
    aucs = []
    for train_idx, test_idx in group_kfold.split(X_all, y_all, groups):
        X_train_cv, X_test_cv = X_all.iloc[train_idx], X_all.iloc[test_idx]
        y_train_cv, y_test_cv = y_all.iloc[train_idx], y_all.iloc[test_idx]

        model = XGBClassifier(**automl.best_config)
        model.fit(X_train_cv, y_train_cv)

        y_pred_proba_cv = model.predict_proba(X_test_cv)[:, 1]
        auc = roc_auc_score(y_test_cv, y_pred_proba_cv)
        aucs.append(auc)

    roc_auc = sum(aucs) / len(aucs)
    return roc_auc


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best Score:", study.best_value)
print("Best Feature Sets:", {k: v for k, v in study.best_params.items()})

[I 2026-01-15 15:55:04,169] A new study created in memory with name: no-name-a4f03d34-41d0-475f-961e-2b097159c626
[I 2026-01-15 15:55:21,607] Trial 0 finished with value: 0.7120757379131982 and parameters: {'hr': True, 'activity': True, 'sleep': True, 'stress': False, 'overnight_hrv': True, 'medical': True, 'therapy': True, 'child_demo': True, 'parent_demo': False, 'temporal': False}. Best is trial 0 with value: 0.7120757379131982.
[I 2026-01-15 15:55:36,075] Trial 1 finished with value: 0.7072883233668128 and parameters: {'hr': False, 'activity': True, 'sleep': False, 'stress': False, 'overnight_hrv': True, 'medical': True, 'therapy': True, 'child_demo': True, 'parent_demo': True, 'temporal': True}. Best is trial 0 with value: 0.7120757379131982.
[I 2026-01-15 15:55:46,210] Trial 2 finished with value: 0.5257960830634276 and parameters: {'hr': False, 'activity': False, 'sleep': False, 'stress': False, 'overnight_hrv': False, 'medical': False, 'therapy': False, 'child_demo': False, 'pa

Best Score: 0.7169362852888242
Best Feature Sets: {'hr': True, 'activity': True, 'sleep': True, 'medical': True, 'therapy': True, 'child_demo': True}


In [32]:
# Get top trials with unique parameters from the study
unique_params = set()
top_unique_trials = []
for t in sorted(
    study.trials,
    key=lambda x: x.value,
    reverse=True,
):
    if len(top_unique_trials) == 10:
        break
    if t.params["hr"] is False:
        continue

    params_tuple = tuple(sorted(t.params.items()))
    if params_tuple not in unique_params and t.value is not None:
        unique_params.add(params_tuple)
        top_unique_trials.append(t)

for idx, trial in enumerate(top_unique_trials):
    print(f"Value={trial.value}, Params={trial.params}")

# Show parameter differences between the top two unique trials
params_0 = top_unique_trials[0].params
params_1 = top_unique_trials[1].params
diff = {k: (params_0[k], params_1[k]) for k in params_0 if params_0[k] != params_1[k]}
print()
print("Parameter differences between top_unique_trials[0] and [1]:", diff)

Value=0.7169362852888242, Params={'hr': True, 'activity': True, 'sleep': True, 'stress': False, 'overnight_hrv': False, 'medical': True, 'therapy': True, 'child_demo': True, 'parent_demo': False, 'temporal': False}
Value=0.7120757379131982, Params={'hr': True, 'activity': True, 'sleep': True, 'stress': False, 'overnight_hrv': True, 'medical': True, 'therapy': True, 'child_demo': True, 'parent_demo': False, 'temporal': False}
Value=0.6957146804840988, Params={'hr': True, 'activity': True, 'sleep': True, 'stress': False, 'overnight_hrv': False, 'medical': False, 'therapy': True, 'child_demo': True, 'parent_demo': True, 'temporal': False}
Value=0.6787986775969839, Params={'hr': True, 'activity': True, 'sleep': False, 'stress': False, 'overnight_hrv': False, 'medical': True, 'therapy': False, 'child_demo': True, 'parent_demo': False, 'temporal': True}
Value=0.6679605784778214, Params={'hr': True, 'activity': True, 'sleep': True, 'stress': False, 'overnight_hrv': True, 'medical': True, 'the

In [ ]:
import shap

fs_subset = ["stress"]
combined_df = pd.concat(
    [
        cleaned_dyads_dfs["index"],
        cleaned_dyads_dfs["response"],
    ]
    + [cleaned_dyads_dfs[fs] for fs in fs_subset],
    axis=1,
)

df_sham = combined_df[combined_df["Arm_Sham"]]
df_treat = combined_df[~combined_df["Arm_Sham"]]

df_train = df_sham
df_test = df_treat
X_train, y_train = prep_X_y(df_train, f"tantrum_within_{window}")
X_test, y_test = prep_X_y(df_test, response_column=f"tantrum_within_{window}")
automl_settings = {
    "time_budget": 5,  # seconds
    # "train_time_limit": 1,  # seconds
    "task": "classification",
    "metric": "log_loss",
    "estimator_list": ["xgboost"],
    # "split_type": time_series_split,
    "early_stop": True,
    "verbose": False,
}
automl = AutoML()
automl.fit(X_train=X_train, y_train=y_train, **automl_settings)
model = XGBClassifier(**automl.best_config)
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC AUC: {roc_auc:.4f}")

# Create SHAP explainer
explainer = shap.Explainer(model)
shap_values = explainer(X_test)
# Note: Bar plot does not accept "group_remaining_features" argument
shap.plots.bar(shap_values, max_display=15)